In [1]:
# Muhammad Usman Nazeer
# I21-0556
# Y
# Assigmnet 2

# libraries included

import time
import resource
import random

# Class made for solving sudoku puzzle
class SudokuSolver:
    def __init__(self, puzzle):
        self.puzzle = puzzle
        self.size = 9
        self.domains = [[set(range(1, 10)) for _ in range(self.size)] for _ in range(self.size)]
        # Initialize the domain for each cell.
        self.initialize_domains()

    # Initialize domains depending on the original puzzle setup.
    def initialize_domains(self):
      i = 0
      while i < self.size:
        j = 0
        while j < self.size:
            if self.puzzle[i][j] != 0:
              # If a cell already has a value, change its domain to include only that value.
                self.domains[i][j] = {self.puzzle[i][j]}
            j += 1
        i += 1

    # Determine whether inserting a number at a certain spot violates Sudoku rules.
    def is_valid(self, row, col, num):
       i = 0
       while i < self.size:
           if self.puzzle[row][i] == num or self.puzzle[i][col] == num:
               return False
           i += 1

       start_row, start_col = 3 * (row // 3), 3 * (col // 3)
       i = 0
       while i < 3:
           j = 0
           while j < 3:
               if self.puzzle[i + start_row][j + start_col] == num:
                  return False
               j += 1
           i += 1
       return True


    def get_empty_cell(self):
        i, j = 0, 0
        # Find the first empty cell in the puzzle.
        while i < self.size:
           while j < self.size:
               if self.puzzle[i][j] == 0:
                  return i, j
               j += 1
        i += 1
        j = 0
        # If no empty cell is found.
        return None, None

    # Function to implement MRV algorithm
    def get_mrv_cell(self):
        # Find the cell with the minimum remaining values
        min_remaining_values = float('inf')
        mrv_cell = None, None
        for i in range(self.size):
            for j in range(self.size):
                if self.puzzle[i][j] == 0:
                    remaining_values = len(self.domains[i][j])
                    if remaining_values < min_remaining_values:
                        min_remaining_values = remaining_values
                        mrv_cell = i, j
        return mrv_cell

    # Function to get degree heuristic values
    def get_degree_heuristic(self, row, col):
        degree = 0
        for i in range(self.size):
            if self.puzzle[row][i] == 0:
                degree += 1
            if self.puzzle[i][col] == 0:
                degree += 1
        start_row, start_col = 3 * (row // 3), 3 * (col // 3)
        for i in range(3):
            for j in range(3):
                if self.puzzle[i + start_row][j + start_col] == 0:
                    degree += 1
        return degree

    # Function to get Least constraining values
    def get_least_constraining_value(self, row, col):
        values_count = {}
        for num in self.domains[row][col]:
            count = 0
            for i in range(self.size):
                if num in self.domains[row][i] or num in self.domains[i][col]:
                    count += 1
            start_row, start_col = 3 * (row // 3), 3 * (col // 3)
            for i in range(3):
                for j in range(3):
                    if num in self.domains[i + start_row][j + start_col]:
                        count += 1
            values_count[num] = count
        return min(values_count, key=values_count.get)

    def is_valid_assignment(self):
       # Check if the current puzzle configuration is valid.
        for i in range(self.size):
            for j in range(self.size):
                num = self.puzzle[i][j]
                if num == 0:
                    return False
                self.puzzle[i][j] = 0
                if not self.is_valid(i, j, num):
                    return False
                self.puzzle[i][j] = num
        return True

    # Apply arc consistency to reduce domains
    def ac3(self):
        queue = [(i, j) for i in range(self.size) for j in range(self.size)]
        while queue:
            i, j = queue.pop(0)
            if self.revise(i, j):
                if not self.domains[i][j]:
                    return False
                neighbors = self.get_neighbors(i, j)
                queue.extend(neighbors)
        return True
    # To revise the domain of cell
    def revise(self, row, col):
        revised = False
        for num in list(self.domains[row][col]):
            if not self.has_consistent_value(row, col, num):
                self.domains[row][col].remove(num)
                revised = True
        return revised

    # To check is placed number violates any rules of game
    def has_consistent_value(self, row, col, num):
        i = 0
        while i < self.size:
            if i != col and num in self.domains[row][i]:
                return True
            if i != row and num in self.domains[i][col]:
                return True
            i += 1

        start_row, start_col = 3 * (row // 3), 3 * (col // 3)
        i = 0
        while i < 3:
            j = 0
            while j < 3:
                if (start_row + i != row or start_col + j != col) and num in self.domains[start_row + i][start_col + j]:
                    return True
                j += 1
            i += 1

        return False

    # Function to generates list of neighboring cells for a given cell

    def get_neighbors(self, row, col):
        neighbors = []
        i = 0
        while i < self.size:
            if i != row:
                neighbors.append((i, col))
            if i != col:
                neighbors.append((row, i))
            i += 1

        start_row, start_col = 3 * (row // 3), 3 * (col // 3)
        i = 0
        while i < 3:
            j = 0
            while j < 3:
                if start_row + i != row or start_col + j != col:
                    neighbors.append((start_row + i, start_col + j))
                j += 1
            i += 1

        return neighbors

    # recursive call for backtracking

    def solve(self):
        return self.backtrack_search()

    # Applying backtrack search algorithm

    def backtrack_search(self):
        if self.is_valid_assignment():
            return self.puzzle
        row, col = self.get_mrv_cell()
        for num in self.domains[row][col]:
            if self.is_valid(row, col, num):
                self.puzzle[row][col] = num
                result = self.backtrack_search()
                if result is not None:
                    return result
                self.puzzle[row][col] = 0
        return None



def get_time_complexity(puzzle):
    start_time = time.time()
    solver = SudokuSolver(puzzle)
    solution = solver.solve()
    if solution:
      for row in solution:
        print(row)
    else:
      print("No solution exists.")
    end_time = time.time()
    elapsed_time = end_time - start_time
    return elapsed_time

def generate_solved_puzzle():
    puzzle = [[(i * 3 + i // 3 + j) % 9 + 1 for j in range(9)] for i in range(9)]
    iterations = 0
    while iterations < 20:
        region = random.randrange(0, 3)
        row1 = random.randrange(region * 3, (region + 1) * 3)
        row2 = random.randrange(region * 3, (region + 1) * 3)
        puzzle[row1], puzzle[row2] = puzzle[row2], puzzle[row1]
        iterations += 1
    puzzle = list(map(list, zip(*puzzle)))
    iterations = 0
    while iterations < 20:
        region = random.randrange(0, 3)
        col1 = random.randrange(region * 3, (region + 1) * 3)
        col2 = random.randrange(region * 3, (region + 1) * 3)
        puzzle[col1], puzzle[col2] = puzzle[col2], puzzle[col1]
        iterations += 1
    return puzzle

def generate_random_puzzle():
    # Generate a random Sudoku puzzle by removing numbers from a solved puzzle.
    solved_puzzle = generate_solved_puzzle()
    puzzle = [[0] * 9 for _ in range(9)]
    for i in range(9):
        for j in range(9):
            puzzle[i][j] = solved_puzzle[i][j]
    iterations = 0
    while iterations < 60:
        i, j = random.randint(0, 8), random.randint(0, 8)
        puzzle[i][j] = 0  # Remove a number from a random cell.
        iterations += 1
    return puzzle

# Generate a random Sudoku puzzle.
random_puzzle = generate_random_puzzle()
for row in random_puzzle:
    print(row)

# Measure space and time complexity of the solver.
print("")
Sudoku_size_bytes = resource.getrusage(resource.RUSAGE_SELF).ru_maxrss / (1024 * 1024)
time_complexity = get_time_complexity(random_puzzle)

print("Memory usage of the Sudoku : {:.2f} MB".format(Sudoku_size_bytes))
print("Time Complexity:", time_complexity, "seconds")
